# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use variables defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from hw3t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    # Track changes to flow out of source and into sink
    flow_into_sink=0
    flow_out_source=0  
    
    #create a residual graph, transformed graph of original graph, add super source and sink
    graph_resid = nx.Graph()
    graph_resid.add_edges_from(graph.edges())
    graph_resid.add_nodes_from(graph.nodes())
    graph_resid.add_node('s*')
    graph_resid.add_node('t*')
    
    for node in list(graph.nodes()):                              # for every node in the graph
        if graph.nodes[node]['demand'] <= 0:                      # if it is a source node
            new_demand=int(-graph.nodes[node]['demand'])          # new demand is equal to negative current demand
            graph_resid.add_edge("s*",node)                       # add edge to the super source
            graph_resid.edges["s*",node]['capacity'] = new_demand # assign the capacity of this edge as the new demand
            flow_out_source = flow_out_source + new_demand        # add this change in flow to flow out of the source
        if graph.nodes[node]['demand'] > 0:                       # if it is a sink node
            new_demand=graph.nodes[node]['demand']                # new demand is equal to current demand
            graph_resid.add_edge(node, "t*")                      # add edge to the super sink
            graph_resid.edges[node, "t*"]['capacity'] = new_demand# assign the capacity of this edge as the new demand
            flow_into_sink = flow_into_sink + new_demand          # add this change in flow to flow into the sink
       
    # If demand constraints can't be satisfied, no feasable circulation
    if flow_out_source != flow_into_sink:
        raise nx.NetworkXUnfeasible('There is no flow satisfying demands')
        
    else: # If demand constraints are satisfied, calculate max flow, return a flow dictionary
        max_flow, all_flows = nx.maximum_flow(graph_resid, 's*', 't*')
        if max_flow != flow_out_source:
            raise nx.NetworkXUnfeasible('There is no flow satisfying demands')
            
        #removing super source and sink, corresponding edges
        for node, edges in all_flows.items():
            if "t*" in edges:  del all_flows[node]['t*']
            if "s*" in edges:  del all_flows[node]['s*']
        del all_flows['t*'] 
        del all_flows['s*']
        del graph_resid
        return all_flows

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    # Store dictionary of the net flow into each node
    new_flow={}
    #set default flow of zero for all nodes
    for node in flow:
        new_flow[node]=0  
    for node in flow:                                     # for every node in the graph
        for edge in flow[node]:                           # for every edge attached to that node            
            new_flow[node]=new_flow[node]-flow[node][edge]# subtract flow on edge from net flow of current node
            new_flow[edge]=new_flow[edge]+flow[node][edge]# add flow on edge to net flow of the edge node
    return new_flow

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)